_inserts one my fav kdot lyric_

**Kendrick Lamar**

Importing necessary libraries

In [3]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

Establishing a Connection with the Spotify API

In [4]:
current_dir = os.path.dirname(os.path.abspath("Kendrick Lamar"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

# Read the config file
config.read(config_path)
            
client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

Function to monitor duration of scrape

In [5]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

Artist Profile harvesting

In [6]:
from spotipy.oauth2 import SpotifyClientCredentials

# Get credentials from the config file
client_id= '31d85b3f430e45caba7b97174cb75137'
client_secret = '5c6b719bcd8743b0ad1f8cd217ad1e1c'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
def collect_spotify_data():
    start_time = timer()
    data = []  # Using a list of dictionaries to collect data

    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist: Kendrick Lamar', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                # Get audio features for track
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    # Handle case where audio features are not available
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                # Get featured artists
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5)  # Retry after a short delay
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break  # Break the loop if there's a persistent error

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print("Elapsed time:", timekeeper(elapsed_time))
    return df_raw

# Collecting data from Spotify API 
df_spotify = collect_spotify_data()

Elapsed time: 3 minutes, 32 seconds


Filtering the artist name column to contain only 'Kendrick Lamar'

In [10]:
kdot = df_spotify[df_spotify['artist_name']=='Kendrick Lamar']
kdot.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
1,Kendrick Lamar,Not Like Us,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,2024-05-04,90,True,0.898,274192,...,-7.001,1,0.0776,0.01070,0.0,0.1410,0.214,101.061,4,[]
3,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,78,True,0.716,386907,...,-7.355,1,0.1220,0.07030,0.0,0.2240,0.344,71.994,4,[Jay Rock]
4,Kendrick Lamar,DNA.,6HZILIRieu8S0iqY8kIKhj,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,79,True,0.638,185947,...,-6.664,1,0.3570,0.00454,0.0,0.0842,0.422,139.913,4,[]
5,Kendrick Lamar,Alright,3iVcZ5G6tvkXZkZKlMpIUs,To Pimp A Butterfly,7ycBtnsMtyVbbwTfJwRjSP,2015-03-16,79,True,0.796,219333,...,-5.974,1,0.2380,0.07420,0.0,0.0827,0.558,110.034,4,[]
6,Kendrick Lamar,LOVE. FEAT. ZACARI.,6PGoSes0D9eUDeeAafB2As,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,81,True,0.800,213400,...,-7.343,1,0.0924,0.26400,0.0,0.1530,0.779,126.058,4,[Zacari]


In [19]:
kdot['album_name'].unique()

array(['Not Like Us', 'good kid, m.A.A.d city', 'DAMN.',
       'To Pimp A Butterfly', 'meet the grahams', 'Section.80',
       'Mr. Morale & The Big Steppers', 'good kid, m.A.A.d city (Deluxe)',
       'i', 'untitled unmastered.', 'Alright', 'Overly Dedicated',
       'DAMN. COLLECTORS EDITION.', 'untitled 07 | levitate', 'Top 30 US',
       'The Heart Part 5', 'Gangsta Bangaz Vol. 1', 'Rainy Day R&B',
       'Rap Gods', 'We Run L.A.', 'Hip Hop Hit List (Vol. 3)',
       'Beaches & Bridges - La 2 da Bay, Vol. 5',
       'The Hate U Give (Original Motion Picture Soundtrack)',
       'West Coast Hip-Hop', 'LoFi Chillhop', 'Best Of 2017',
       'Hip Hop Hit List - Vol. IV', 'Spring Party 2018',
       '2018 Award Nominees'], dtype=object)

Categorizing the extract according to what type of release

In [24]:
studio_albums = ['Section.80',
                 'good kid, m.A.A.d city',
                 'To Pimp A Butterfly',                 
                 'DAMN.',
                 'DAMN.COLLECTORS EDITION.',
                 'Mr. Morale & The Big Steppers']

compilation_album = ['untitled unmastered.']

deluxe_album = ['good kid, m.A.A.d city (Deluxe)']

mixtape = ['Overly Dedicated']

album_condition = kdot['album_name'].isin(studio_albums)
compilations = kdot['album_name'].isin(compilation_album)
deluxe = kdot['album_name'].isin(deluxe_album)
mixtape_mapping = kdot['album_name'].isin(mixtape)

conditions = [album_condition, compilations, deluxe, mixtape_mapping]

choices = ['Studio Album', 'Compilation', 'Deluxe Album', 'Mixtape']

kdot['type'] = np.select(conditions, choices, default= 'Single')

kdot

C:\Users\HP\AppData\Local\Temp\ipykernel_13804\1279416075.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kdot['type'] = np.select(conditions, choices, default= 'Single')


,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,type
1,Kendrick Lamar,Not Like Us,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,2024-05-04,90,True,0.898,274192,...,1,0.0776,0.010700,0.000000,0.1410,0.214,101.061,4,[],Single
3,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,78,True,0.716,386907,...,1,0.1220,0.070300,0.000000,0.2240,0.344,71.994,4,[Jay Rock],Studio Album
4,Kendrick Lamar,DNA.,6HZILIRieu8S0iqY8kIKhj,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,79,True,0.638,185947,...,1,0.3570,0.004540,0.000000,0.0842,0.422,139.913,4,[],Studio Album
5,Kendrick Lamar,Alright,3iVcZ5G6tvkXZkZKlMpIUs,To Pimp A Butterfly,7ycBtnsMtyVbbwTfJwRjSP,2015-03-16,79,True,0.796,219333,...,1,0.2380,0.074200,0.000000,0.0827,0.558,110.034,4,[],Studio Album
6,Kendrick Lamar,LOVE. FEAT. ZACARI.,6PGoSes0D9eUDeeAafB2As,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,81,True,0.800,213400,...,1,0.0924,0.264000,0.000000,0.1530,0.779,126.058,4,[Zacari],Studio Album
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Kendrick Lamar,Institutionalized,5HPYKvFGW25YpGQrtYTO9o,West Coast Hip-Hop,4qCycaVc21EEOCHysLXHYV,2018-10-26,5,True,0.539,271773,...,0,0.4510,0.112000,0.000001,0.1290,0.563,81.114,4,"[Bilal, Anna Wise, Snoop Dogg]",Single
384,Kendrick Lamar,DNA.,7Bja4UJhhS8pZQtmU8ia6p,Best Of 2017,197YY02cK93Yjrtpa1FrMY,2017-12-08,8,True,0.639,185947,...,1,0.3490,0.003930,0.000000,0.0783,0.391,139.902,4,[],Single
420,Kendrick Lamar,ELEMENT.,3m7B5vvKuFJ3gMARIzyKef,Hip Hop Hit List - Vol. IV,3FDzQi8m9VmWcJaxLB0sSp,2018-03-30,4,True,0.763,208733,...,1,0.4680,0.223000,0.000000,0.2310,0.486,189.979,4,[],Single
444,Kendrick Lamar,LOVE.,2QroLtVYUvpQ8i08pdqn1n,Spring Party 2018,6Oe7PIFiEq9k1d4zzftFN7,2018-03-23,7,True,0.801,214449,...,1,0.0958,0.258000,0.000000,0.1320,0.773,126.049,4,[Zacari],Single
